In [1]:
import os
import copy

from pyciemss.PetriNetODE.base import MiraPetriNetODESystem, ScaledBetaNoisePetriNetODESystem
from pyciemss.PetriNetODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
from pyciemss.Ensemble.base import EnsembleSystem
import pyciemss

from pyciemss.PetriNetODE.interfaces import load_petri_model
from pyciemss.Ensemble.interfaces import setup_model, reset_model, intervene, sample, calibrate, optimize

In [2]:
MIRA_PATH = "test/models/april_ensemble_demo/"

filename1 = "BIOMD0000000955_template_model.json"
filename1 = os.path.join(MIRA_PATH, filename1)
model1 = load_petri_model(filename1, add_uncertainty=True)


# TODO: put this into the interfaces
start_state1 = {k[0]: v.data['initial_value'] for k, v in model1.G.variables.items()}

model1

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	gamma = Uniform(low: 0.41040000319480896, high: 0.5016000270843506),
	delta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	alpha = Uniform(low: 0.5130000114440918, high: 0.6269999742507935),
	epsilon = Uniform(low: 0.15389999747276306, high: 0.18809999525547028),
	zeta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	XXlambdaXX = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	eta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	rho = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	theta = Uniform(low: 0.33390000462532043, high: 0.4081000089645386),
	kappa = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	mu = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	nu = Uniform(low: 0.024299999698996544, high: 0.02969999983906746),
	xi = Uniform(low: 0.015300000086426735, high: 

In [3]:
filename2 = "BIOMD0000000960_template_model.json"
filename2 = os.path.join(MIRA_PATH, filename2)
model2 = load_petri_model(filename2, add_uncertainty=True)


# TODO: put this into the interfaces
start_state2 = {k[0]: v.data['initial_value'] for k, v in model2.G.variables.items()}

model2

ScaledBetaNoisePetriNetODESystem(
	mira_param_0 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_1 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_2 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_3 = Uniform(low: 0.02098800055682659, high: 0.025652000680565834),
	mira_param_4 = Uniform(low: 0.37501201033592224, high: 0.45834800601005554),
	mira_param_5 = Uniform(low: 0.4526999890804291, high: 0.5533000230789185),
	mira_param_6 = Uniform(low: 0.23669999837875366, high: 0.28929999470710754),
	mira_param_7 = Uniform(low: 0.0027000000700354576, high: 0.0032999999821186066),
	mira_param_8 = Uniform(low: 1.4759999513626099, high: 1.8040000200271606),
	mira_param_9 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_10 = Uniform(low: 0.007199999876320362, high: 0.008799999952316284),
	mira_param_11 = Uniform(low: 0.12690000236034393, high: 0.1551000028848648),
	pseudocount = 1.0
)

In [4]:
from math import isclose

solution_ratio = start_state2['Infectious'] / start_state1['Infected']
solution_mapping1 = lambda x : {"Infected": x["Infected"]}
solution_mapping2 = lambda x: {"Infected": x["Infectious"] / solution_ratio}

# Assert that all of the variables in the solution mappings are the same.
assert(set(solution_mapping1(start_state1).keys()) == set(solution_mapping2(start_state2).keys()))

# Assert that the solution mappings are correct.
assert(isclose(solution_mapping1(start_state1)["Infected"], solution_mapping2(start_state2)["Infected"]))

In [5]:
# # JEREMY LOOK HERE.

# filename3 = "BIOMD0000000983_template_model.json"
# filename3 = os.path.join(MIRA_PATH, filename3)
# model3 = load_petri_model(filename3, add_uncertainty=True)

# # TODO: put this into the interfaces
# start_state3 = {k[0]: v.data['initial_value'] for k, v in model3.G.variables.items()}

# model3

In [7]:
# Setup the Ensemble

models = [model1, model2]
weights = [0.5, 0.5]
start_time = 0.0

start_states = [start_state1, start_state2]
solution_mappings = [solution_mapping1, solution_mapping2]

total_population = 1.0
dirichlet_concentration = 10.0

ensemble = setup_model(models, 
                       weights, 
                       solution_mappings, 
                       start_time, 
                       start_states, 
                       total_population, 
                       dirichlet_concentration=dirichlet_concentration)
ensemble

Ensemble of 2 models. 

 	Dirichlet Alpha: tensor([5., 5.]). 

 	Models: [ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	gamma = Uniform(low: 0.41040000319480896, high: 0.5016000270843506),
	delta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	alpha = Uniform(low: 0.5130000114440918, high: 0.6269999742507935),
	epsilon = Uniform(low: 0.15389999747276306, high: 0.18809999525547028),
	zeta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	XXlambdaXX = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	eta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	rho = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	theta = Uniform(low: 0.33390000462532043, high: 0.4081000089645386),
	kappa = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	mu = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	nu = Uniform(low: 0.024299999698996544, h

In [14]:
# Sample from the Ensemble

timepoints = [1.0, 5.0, 10.0]
num_samples = 2
ensemble_solution = sample(ensemble, timepoints, num_samples)
ensemble_solution


{'model_0/beta': tensor([0.0115, 0.0103]),
 'model_0/gamma': tensor([0.4975, 0.4608]),
 'model_0/delta': tensor([0.0115, 0.0112]),
 'model_0/alpha': tensor([0.5387, 0.5770]),
 'model_0/epsilon': tensor([0.1776, 0.1634]),
 'model_0/zeta': tensor([0.1241, 0.1310]),
 'model_0/XXlambdaXX': tensor([0.0369, 0.0358]),
 'model_0/eta': tensor([0.1155, 0.1204]),
 'model_0/rho': tensor([0.0308, 0.0362]),
 'model_0/theta': tensor([0.3523, 0.3512]),
 'model_0/kappa': tensor([0.0163, 0.0155]),
 'model_0/mu': tensor([0.0174, 0.0185]),
 'model_0/nu': tensor([0.0272, 0.0292]),
 'model_0/xi': tensor([0.0185, 0.0158]),
 'model_0/tau': tensor([0.0093, 0.0109]),
 'model_0/sigma': tensor([0.0187, 0.0158]),
 'model_1/mira_param_0': tensor([0.0848, 0.0463]),
 'model_1/mira_param_1': tensor([0.0821, 0.0246]),
 'model_1/mira_param_2': tensor([0.0865, 0.0228]),
 'model_1/mira_param_3': tensor([0.0240, 0.0213]),
 'model_1/mira_param_4': tensor([0.4252, 0.4177]),
 'model_1/mira_param_5': tensor([0.4600, 0.4762]),


In [15]:
data = [(1.1, {"Infected": 0.003}), (1.2, {"Infected": 0.005})]

# TODO: increase the number of iterations
inferred_parameters = calibrate(ensemble, data, num_iterations=1000, verbose=True)

iteration 0: loss = 85.91385185718536
iteration 25: loss = 61.73275029659271
iteration 50: loss = 46.769694209098816
iteration 75: loss = 30.32486641407013


In [16]:
posterior_predictive = sample(ensemble, timepoints, num_samples, inferred_parameters)
posterior_predictive

{'model_0/beta': tensor([0.0110, 0.0108]),
 'model_0/gamma': tensor([0.4769, 0.4470]),
 'model_0/delta': tensor([0.0107, 0.0115]),
 'model_0/alpha': tensor([0.5843, 0.5954]),
 'model_0/epsilon': tensor([0.1744, 0.1641]),
 'model_0/zeta': tensor([0.1169, 0.1226]),
 'model_0/XXlambdaXX': tensor([0.0347, 0.0328]),
 'model_0/eta': tensor([0.1277, 0.1211]),
 'model_0/rho': tensor([0.0327, 0.0366]),
 'model_0/theta': tensor([0.4003, 0.3625]),
 'model_0/kappa': tensor([0.0183, 0.0183]),
 'model_0/mu': tensor([0.0167, 0.0178]),
 'model_0/nu': tensor([0.0246, 0.0273]),
 'model_0/xi': tensor([0.0175, 0.0171]),
 'model_0/tau': tensor([0.0101, 0.0104]),
 'model_0/sigma': tensor([0.0175, 0.0177]),
 'model_1/mira_param_0': tensor([0.0489, 0.0564]),
 'model_1/mira_param_1': tensor([0.0288, 0.0380]),
 'model_1/mira_param_2': tensor([0.0527, 0.0793]),
 'model_1/mira_param_3': tensor([0.0223, 0.0231]),
 'model_1/mira_param_4': tensor([0.4205, 0.4055]),
 'model_1/mira_param_5': tensor([0.4896, 0.4580]),


## MIRA Explorations below

In [10]:
import json
import mira

filename1 = "BIOMD0000000955_template_model.json"
filename1 = os.path.join(MIRA_PATH, filename1)

with open(filename1, "r") as f:
    model_json1 = json.load(f)

template1 = mira.metamodel.TemplateModel.from_json(model_json1)
model1 = mira.modeling.Model(template1)

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

template3 = mira.metamodel.TemplateModel.from_json(model_json3)
model3 = mira.modeling.Model(template3)
# G = pyciemss.PetriNetODE.base.PetriNet.from_mira(filename3)

# model3 = MiraPetriNetODESystem.from_mira(filename3)
# model3 = ScaledBetaNoisePetriNetODESystem.from_mira(filename3)
# model3

In [11]:
from mira.metamodel.ops import simplify_rate_laws, aggregate_parameters

In [8]:
template1.parameters

{'alpha': Parameter(name='alpha', identifiers={}, context={}, value=0.57, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.45599999999999996, 'maximum': 0.6839999999999999})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'gamma': Parameter(name='gamma', identifiers={}, context={}, value=0.456, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472})),
 'delta': Parameter(name='delta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'epsilon': Parameter(name='epsilon', identifiers={}, context={}, value=0.171, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.1368, 'maximum': 0.20520000000000002})),
 'theta': Parameter(name='theta', iden

In [16]:
new_template = simplify_rate_laws(aggregate_parameters(template1))
# new_template = simplify_rate_laws(template1)
new_template.parameters



{'alpha': Parameter(name='alpha', identifiers={}, context={}, value=0.57, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.45599999999999996, 'maximum': 0.6839999999999999})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'gamma': Parameter(name='gamma', identifiers={}, context={}, value=0.456, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472})),
 'delta': Parameter(name='delta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'epsilon': Parameter(name='epsilon', identifiers={}, context={}, value=0.171, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.1368, 'maximum': 0.20520000000000002})),
 'theta': Parameter(name='theta', iden

In [12]:

template3.parameters

{'f': Parameter(name='f', identifiers={}, context={}, value=0.2, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.16, 'maximum': 0.24000000000000002})),
 'n': Parameter(name='n', identifiers={}, context={}, value=0.5, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.4, 'maximum': 0.6})),
 'q': Parameter(name='q', identifiers={}, context={}, value=0.83, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.6639999999999999, 'maximum': 0.996})),
 'sigma': Parameter(name='sigma', identifiers={}, context={}, value=0.2, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.16, 'maximum': 0.24000000000000002})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=2.115e-08, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 1.6920000000000002e-08, 'maximum': 2.538e-08})),
 'mu': Parameter(name='mu', identifiers={}, context={}, value=0.2, distribution=Distribu

In [22]:
from mira.metamodel.ops import aggregate_parameters

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

template3 = mira.metamodel.TemplateModel.from_json(model_json3)
model3 = mira.modeling.Model(template3)

new_template = aggregate_parameters(template3)
new_model = mira.modeling.Model(new_template)
new_model.parameters

{'mira_param_0': <mira.modeling.ModelParameter at 0x7fee91669960>,
 'mira_param_1': <mira.modeling.ModelParameter at 0x7fee91669de0>,
 'mira_param_2': <mira.modeling.ModelParameter at 0x7fee91669d50>,
 'mira_param_3': <mira.modeling.ModelParameter at 0x7fee916694b0>,
 'mira_param_4': <mira.modeling.ModelParameter at 0x7fee91669450>,
 'mira_param_5': <mira.modeling.ModelParameter at 0x7fee91669360>,
 'mira_param_6': <mira.modeling.ModelParameter at 0x7fee91669330>,
 'mira_param_7': <mira.modeling.ModelParameter at 0x7fee91669240>,
 'mira_param_8': <mira.modeling.ModelParameter at 0x7fee91669210>,
 'XXlambdaXX': <mira.modeling.ModelParameter at 0x7fee91669f30>,
 'theta': <mira.modeling.ModelParameter at 0x7fee91669150>,
 (('Quarantined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C71902')),
  ('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  'NaturalConversion',
  'rate'): <mira.modeling.ModelParameter at 0x7fee916690f0>,
 'mira_para

In [26]:
model1.parameters

{'beta': <mira.modeling.ModelParameter at 0x7fcb7c5388e0>,
 'gamma': <mira.modeling.ModelParameter at 0x7fcb7c53bc10>,
 'delta': <mira.modeling.ModelParameter at 0x7fcb7c53a050>,
 'alpha': <mira.modeling.ModelParameter at 0x7fcb7c5e9570>,
 'epsilon': <mira.modeling.ModelParameter at 0x7fcb7c5eb430>,
 'zeta': <mira.modeling.ModelParameter at 0x7fcb7c5eac80>,
 'XXlambdaXX': <mira.modeling.ModelParameter at 0x7fcb7c5eabc0>,
 'eta': <mira.modeling.ModelParameter at 0x7fcb7c5eab00>,
 'rho': <mira.modeling.ModelParameter at 0x7fcb7c5eaa70>,
 'theta': <mira.modeling.ModelParameter at 0x7fcb7c5ea9e0>,
 'kappa': <mira.modeling.ModelParameter at 0x7fcb7c5ea950>,
 'mu': <mira.modeling.ModelParameter at 0x7fcb7c5ea890>,
 'nu': <mira.modeling.ModelParameter at 0x7fcb7c5ea7d0>,
 'xi': <mira.modeling.ModelParameter at 0x7fcb7c5ea740>,
 'tau': <mira.modeling.ModelParameter at 0x7fcb7c5ea680>,
 'sigma': <mira.modeling.ModelParameter at 0x7fcb7c5ea5c0>}

In [44]:
model3.transitions

{(('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Exposed', ('identity', 'apollosv:00000154')),
  ('Infected_reported',
   ('identity', 'ido:0000511'),
   ('diagnosis', 'ncit:C15220')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8d00>,
 (('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Exposed', ('identity', 'apollosv:00000154')),
  ('Infected_unreported',
   ('identity', 'ido:0000511'),
   ('diagnosed', 'ncit:C113725')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8c10>,
 (('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Quarantined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C71902')),
  ('Infected_reported',
   ('identity', 'ido:0000511'),
   ('diagnosis', 'ncit:C15220')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8b20>,
 (('Susceptible_unconfined',
   ('ide

In [42]:
from mira.modeling.ode import OdeModel

ode1 = OdeModel(model1)

ode1.set_parameters({'beta': 0.5})

ode1.kinetics
ode1.get_rhs()

<function mira.modeling.ode.OdeModel.get_rhs.<locals>.rhs(t, y)>

In [35]:


list(model3.transitions.values())[0].template

ControlledConversion(rate_law=Infected_reported*Susceptible_unconfined*beta*n*(1.0 - sigma), type='ControlledConversion', controller=Concept(name='Infected_reported', identifiers={'ido': '0000511'}, context={'diagnosis': 'ncit:C15220'}), subject=Concept(name='Susceptible_unconfined', identifiers={'ido': '0000514'}, context={'quarantined': 'ncit:C68851'}), outcome=Concept(name='Exposed', identifiers={'apollosv': '00000154'}, context={}), provenance=[])

In [13]:
import json
import mira

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

example_template = mira.metamodel.TemplateModel.from_json(model_json3)

example_model = mira.modeling.Model(example_template)

In [14]:
for p, v in example_template.parameters.items():
    print(p, v.value)
    

f 0.2
n 0.5
q 0.83
sigma 0.2
beta 2.115e-08
mu 0.2
eta_r 0.1
eta_u 0.25
theta 0.07142857
_1 1.0
XXlambdaXX 0.3
h 0.05


In [ ]:
for p, v in example_template.parameters.items():
    print(p, v.value)

In [20]:
class Test1():
    def __init__(self):
        self.x = 0

    def test_func(self):
        print(self.x)
        self.x += 1
        print(self.x)
        
        

class Test2():
    def __init__(self, t2):
        self.x = 1
        self.t2 = t2

    def test_func(self):
        return self.t2.__class__.test_func(self)

In [21]:
t1 = Test1()
t2 = Test2(t1)
t2.test_func()

1
2


In [13]:
t1.__class__.test_func

<function __main__.Test1.test_func(self)>

In [20]:
# Get the class of the object
class_ = t2.test_func.__self__.__class__

In [21]:
class_

__main__.Test1